In [188]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [189]:
import ast
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [190]:
movies=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [191]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [192]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [193]:
movies.shape

(4803, 20)

In [194]:
movies=movies.merge(credits,on='title')
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [195]:
movies=movies[['id','title','runtime','genres','budget','revenue','cast','crew']]
movies.head()

,id,title,runtime,genres,budget,revenue,cast,crew
0,19995,Avatar,162.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",237000000,2787965087,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,169.0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",300000000,961000000,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,148.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",245000000,880674609,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,165.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",250000000,1084939099,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,132.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",260000000,284139100,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [196]:
movies.dropna(inplace=True)

In [197]:
movies.drop_duplicates(inplace=True)

In [198]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [199]:
movies['genres']=movies['genres'].apply(convert)
movies.head()

,id,title,runtime,genres,budget,revenue,cast,crew
0,19995,Avatar,162.0,"[Action, Adventure, Fantasy, Science Fiction]",237000000,2787965087,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,169.0,"[Adventure, Fantasy, Action]",300000000,961000000,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,148.0,"[Action, Adventure, Crime]",245000000,880674609,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,165.0,"[Action, Crime, Drama, Thriller]",250000000,1084939099,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,132.0,"[Action, Adventure, Science Fiction]",260000000,284139100,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [200]:
def convert3(text):
    L=[]
    counter=0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L

In [201]:
movies['cast']=movies['cast'].apply(convert)
movies.head()

,id,title,runtime,genres,budget,revenue,cast,crew
0,19995,Avatar,162.0,"[Action, Adventure, Fantasy, Science Fiction]",237000000,2787965087,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,169.0,"[Adventure, Fantasy, Action]",300000000,961000000,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,148.0,"[Action, Adventure, Crime]",245000000,880674609,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,165.0,"[Action, Crime, Drama, Thriller]",250000000,1084939099,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,132.0,"[Action, Adventure, Science Fiction]",260000000,284139100,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [202]:
def fetch_director(text):
    L=[]
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [203]:
movies['crew']=movies['crew'].apply(fetch_director)

In [204]:
def collapse(L):
    L1=[]
    for i in L:
        L1.append(i.replace("",""))
    return L1

In [205]:
movies['cast']=movies['cast'].apply(collapse)
movies['crew']=movies['crew'].apply(collapse)
movies['genres']=movies['genres'].apply(collapse)


In [206]:
print(movies['cast'])

0       [Sam Worthington, Zoe Saldana, Sigourney Weave...
1       [Johnny Depp, Orlando Bloom, Keira Knightley, ...
2       [Daniel Craig, Christoph Waltz, Léa Seydoux, R...
3       [Christian Bale, Michael Caine, Gary Oldman, A...
4       [Taylor Kitsch, Lynn Collins, Samantha Morton,...
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805    [Edward Burns, Kerry Bishé, Marsha Dietlein, C...
4806    [Eric Mabius, Kristin Booth, Crystal Lowe, Geo...
4807    [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldm...
Name: cast, Length: 4807, dtype: object


In [207]:
import pandas as pd

# สร้างลิสต์ของข้อมูลที่มีลิสต์ของนักแสดง
data = movies['cast']

actor_0 = []
actor_1 = []
actor_2 = []
actor_3 = []
actor_4 = []

for cast in range(5):
    actors = data[cast][0:5]
    separated_cast = [actor.split(", ")[0] for actor in actors]
    actor_0.append(separated_cast[0])
    actor_1.append(separated_cast[1])
    actor_2.append(separated_cast[2])
    actor_3.append(separated_cast[3])
    actor_4.append(separated_cast[4])

# สร้าง DataFrame จากลิสต์ของนักแสดง
actor_data = pd.DataFrame({
    'Actor1': actor_0,
    'Actor2': actor_1,
    'Actor3': actor_2,
    'Actor4': actor_3,
    'Actor5': actor_4
})

# ใช้ apply() เพื่อเรียกใช้ฟังก์ชัน remove_brackets บนทุกคอลัมน์ของ DataFrame
actor_data = actor_data.apply(lambda x: x.map(remove_brackets))

# พิมพ์ผลลัพธ์ในรูปแบบของตาราง
print(actor_data)

            Actor1           Actor2            Actor3             Actor4  \
0  Sam Worthington      Zoe Saldana  Sigourney Weaver       Stephen Lang   
1      Johnny Depp    Orlando Bloom   Keira Knightley  Stellan Skarsgård   
2     Daniel Craig  Christoph Waltz       Léa Seydoux      Ralph Fiennes   
3   Christian Bale    Michael Caine       Gary Oldman      Anne Hathaway   
4    Taylor Kitsch     Lynn Collins   Samantha Morton       Willem Dafoe   

                Actor5  
0   Michelle Rodriguez  
1         Chow Yun-fat  
2      Monica Bellucci  
3            Tom Hardy  
4  Thomas Haden Church  


In [208]:
import pandas as pd

# สร้างลิสต์ของข้อมูลที่มีลิสต์ของนักแสดง
data = movies['crew']

director_0 = []


for cast in range(5):
    directors = data[cast]
    separated_cast = [director.split(", ")[0] for director in directors]
    director_0.append(separated_cast[0])


# สร้าง DataFrame จากลิสต์ของนักแสดง
director_data = pd.DataFrame({
    'director': director_0,

})

# ใช้ apply() เพื่อเรียกใช้ฟังก์ชัน remove_brackets บนทุกคอลัมน์ของ DataFrame
director_data = director_data.apply(lambda x: x.map(remove_brackets))

# พิมพ์ผลลัพธ์ในรูปแบบของตาราง
print(director_data)

            director
0      James Cameron
1     Gore Verbinski
2         Sam Mendes
3  Christopher Nolan
4     Andrew Stanton


In [209]:
movies.drop(columns=['cast'], axis=1, inplace=True)
movies.drop(columns=['crew'], axis=1, inplace=True)

# พิมพ์ผลลัพธ์ที่ได้
print(movies[0:5])

       id                                     title  runtime  \
0   19995                                    Avatar    162.0   
1     285  Pirates of the Caribbean: At World's End    169.0   
2  206647                                   Spectre    148.0   
3   49026                     The Dark Knight Rises    165.0   
4   49529                               John Carter    132.0   

                                          genres     budget     revenue  
0  [Action, Adventure, Fantasy, Science Fiction]  237000000  2787965087  
1                   [Adventure, Fantasy, Action]  300000000   961000000  
2                     [Action, Adventure, Crime]  245000000   880674609  
3               [Action, Crime, Drama, Thriller]  250000000  1084939099  
4           [Action, Adventure, Science Fiction]  260000000   284139100  


In [210]:
movies = pd.concat([movies, actor_data, director_data], axis=1, ignore_index=False)

# พิมพ์ผลลัพธ์ที่ได้
print(movies)

          id                                     title  runtime  \
0      19995                                    Avatar    162.0   
1        285  Pirates of the Caribbean: At World's End    169.0   
2     206647                                   Spectre    148.0   
3      49026                     The Dark Knight Rises    165.0   
4      49529                               John Carter    132.0   
...      ...                                       ...      ...   
4804    9367                               El Mariachi     81.0   
4805   72766                                 Newlyweds     85.0   
4806  231617                 Signed, Sealed, Delivered    120.0   
4807  126186                          Shanghai Calling     98.0   
4808   25975                         My Date with Drew     90.0   

                                             genres     budget     revenue  \
0     [Action, Adventure, Fantasy, Science Fiction]  237000000  2787965087   
1                      [Adventure, Fant

In [211]:
movies.head()

,id,title,runtime,genres,budget,revenue,Actor1,Actor2,Actor3,Actor4,Actor5,director
0,19995,Avatar,162.0,"[Action, Adventure, Fantasy, Science Fiction]",237000000,2787965087,Sam Worthington,Zoe Saldana,Sigourney Weaver,Stephen Lang,Michelle Rodriguez,James Cameron
1,285,Pirates of the Caribbean: At World's End,169.0,"[Adventure, Fantasy, Action]",300000000,961000000,Johnny Depp,Orlando Bloom,Keira Knightley,Stellan Skarsgård,Chow Yun-fat,Gore Verbinski
2,206647,Spectre,148.0,"[Action, Adventure, Crime]",245000000,880674609,Daniel Craig,Christoph Waltz,Léa Seydoux,Ralph Fiennes,Monica Bellucci,Sam Mendes
3,49026,The Dark Knight Rises,165.0,"[Action, Crime, Drama, Thriller]",250000000,1084939099,Christian Bale,Michael Caine,Gary Oldman,Anne Hathaway,Tom Hardy,Christopher Nolan
4,49529,John Carter,132.0,"[Action, Adventure, Science Fiction]",260000000,284139100,Taylor Kitsch,Lynn Collins,Samantha Morton,Willem Dafoe,Thomas Haden Church,Andrew Stanton


Note: you may need to restart the kernel to use updated packages.


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/Test/movies_by_KhoaOat.csv'